In [ ]:
import pandas as pd

# Lees de datasets in
lifeexp_df = pd.read_csv("lifeexp.csv") #verander naar naam van het bestand mortality rate dataset
mortality_df = pd.read_csv("mortality.csv")  #verander naar naam van het bestand life expectancy dataset
gdp_per_cap_df = pd.read_csv("GDPpercapita.csv", skiprows=4) #verander naar naam van het bestand GDP per country dataset



# Economische groei en volksgezondheid in mondiale context

Economische groei wordt vaak gezien als motor voor maatschappelijke vooruitgang. Meer welvaart zou leiden tot betere zorg, voeding en leefomstandigheden en dus tot een gezondere bevolking. Maar hoe sterk is die relatie werkelijk?



Door datasets over GDP, levensverwachting en sterfte met elkaar te verbinden, kunnen we trends ontdekken, afwijkingen signaleren en kritische vragen stellen. In plaats van een eenduidige verklaring te zoeken, proberen we inzicht te geven in de variatie en complexiteit die ten grondslag ligt aan mondiale gezondheidsstatistieken.



**Welvaart als motor voor levensverwachting?**

Rijkere landen zouden betere gezondheidszorgsystemen, hygiëne, voeding en toegang tot medicijnen kunnen bekostigen. Maar in hoeverre klopt deze veronderstelling wanneer we wereldwijde data over een langere periode analyseren? En belangrijker nog: toont de relatie tussen economische groei en gezondheid universele vooruitgang, of zien we ook grenzen aan het effect van rijkdom op levensduur?

Met deze vragen in gedachten kijken we naar de relatie tussen het bruto binnenlands product (GDP) per capita en de gemiddelde levensverwachting van mannen tussen 1960 en 2022.

In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np

# Inladen van data
life_df = pd.read_csv("lifeexp.csv")
gdp_df = pd.read_csv("GDPpercapita.csv", skiprows=4)

# Filter en hernoemen
life_df = life_df[(life_df["SEX_NAME"] == "Male") & (life_df["UNIT_MEASURE_NAME"].str.contains("year", case=False))]
life_clean = life_df[["REF_AREA_NAME", "TIME_PERIOD", "OBS_VALUE"]].rename(
    columns={"REF_AREA_NAME": "Country", "TIME_PERIOD": "Year", "OBS_VALUE": "LifeExpectancy"}
)

# GDP data herschikken
gdp_long = gdp_df.melt(id_vars=["Country Name"], var_name="Year", value_name="GDP_per_capita")
gdp_long = gdp_long.rename(columns={"Country Name": "Country"})
gdp_long["Year"] = pd.to_numeric(gdp_long["Year"], errors="coerce")

# Merge datasets
merged = pd.merge(life_clean, gdp_long, on=["Country", "Year"], how="inner")
merged = merged.dropna(subset=["GDP_per_capita", "LifeExpectancy"])
merged["GDP_per_capita"] = pd.to_numeric(merged["GDP_per_capita"], errors="coerce")
merged["Size"] = np.sqrt(merged["GDP_per_capita"]) / np.sqrt(merged["GDP_per_capita"].max()) * 80

# Scatterplot
fig = px.scatter(
    merged,
    x="GDP_per_capita",
    y="LifeExpectancy",
    animation_frame="Year",
    animation_group="Country",
    size="Size",
    color="LifeExpectancy",
    hover_name="Country",
    log_x=True,
    size_max=60,
    range_x=[100, 100000],
    range_y=[30, 90],
    color_continuous_scale="Viridis",
    labels={
        "GDP_per_capita": "GDP per capita (USD)",
        "LifeExpectancy": "Levensverwachting (jaren)"
    },
    title="Relatie tussen levensverwachting en GDP per capita (mannen)"
)

# Extra tickwaarden voor de x-as
fig.update_layout(
    paper_bgcolor="#FFF9E3",
    plot_bgcolor="#FFF9E3",
    font=dict(family="Arial", size=13),
    title_x=0.5,
    xaxis=dict(
        tickvals=[100, 300, 1000, 3000, 10000, 30000, 100000],
        ticktext=["100", "300", "1k", "3k", "10k", "30k", "100k"]
    )
)

fig.add_annotation(
    text="Deze grafiek toont de relatie tussen het bruto nationaal inkomen per hoofd van de bevolking (GDP per capita) en de levensverwachting van mannen wereldwijd.",
    xref="paper", yref="paper",
    x=0.20, y=-0.25,
    showarrow=False,
    font=dict(size=12),
    align="center"
)

fig.add_annotation(
    text="Hogere inkomens gaan vaak gepaard met een hogere levensverwachting, al zijn er duidelijke regionale verschillen en uitzonderingen zichtbaar over de tijd.",
    xref="paper", yref="paper",
    x=0.20, y=-0.30,
    showarrow=False,
    font=dict(size=12),
    align="center"
)

fig.show()


De grafiek laat zien dat een hoger GDP per capita meestal samengaat met een hogere levensverwachting bij mannen. Linksonder bevinden zich landen met lage inkomens en korte levensduur, terwijl rijke landen rechtsboven te vinden zijn. Opvallend is dat boven een bepaald inkomensniveau (±10.000–30.000 dollar) de winst in levensverwachting afvlakt. Er lijkt dus een 'verzadigingspunt' te bestaan. Extra economische groei levert dan minder gezondheidswinst op. Onderzoek van Lutz en Kebede (2018) ondersteunt dit idee en toont aan dat onderwijs een sterkere en duurzamere voorspeller is van levensverwachting dan GDP, waardoor de traditionele Preston-curve mogelijk herzien moet worden. Er lijkt ook sprake van een drempeleffect: pas bij een GDP van circa 200–300 begint de levensverwachting merkbaar toe te nemen, terwijl die daarvoor relatief stabiel blijft, een patroon dat ook wordt bevestigd door Niu Et al. (2021) in een studie over China, waar economische groei pas na een bepaald welvaartsniveau tot gezondheidswinst leidt. De animatie toont bovendien dat deze vooruitgang ongelijk verdeeld is: rijke landen liepen al voorop en bouwen hun voorsprong verder uit, terwijl lage-inkomenslanden pas recent vooruitgang tonen. De wereldwijde gezondheidskloof blijft daarmee zichtbaar.


**Gezondheid binnen regio’s: ongelijkheid voorbij landsgrenzen**

Deze visualisatie toont de gemiddelde levensverwachting per land, met de mogelijkheid om op regio te filteren. Zo kunnen we binnen regio’s de spreiding van levensverwachting analyseren en nagaan of welvaart en geografische nabijheid voldoende zijn om gezondheidsverschillen te verklaren.


In [ ]:
import pandas as pd
import plotly.graph_objects as go

# CSV inladen
lifeexp_df = pd.read_csv("lifeexp.csv")
lifeexp_df = lifeexp_df[(lifeexp_df["SEX_NAME"] == "Male") & (lifeexp_df["UNIT_MEASURE_NAME"].str.lower().str.contains("year"))]

# Voeg hier je volledige region_mapping en region_colors dicts in
region_mapping = {
    'Angola': 'Africa Eastern and Southern', 'Botswana': 'Africa Eastern and Southern',
    'Burundi': 'Africa Eastern and Southern', 'Comoros': 'Africa Eastern and Southern',
    'Djibouti': 'Africa Eastern and Southern', 'Eritrea': 'Africa Eastern and Southern',
    'Eswatini': 'Africa Eastern and Southern', 'Ethiopia': 'Africa Eastern and Southern',
    'Kenya': 'Africa Eastern and Southern', 'Lesotho': 'Africa Eastern and Southern',
    'Madagascar': 'Africa Eastern and Southern', 'Malawi': 'Africa Eastern and Southern',
    'Mauritius': 'Africa Eastern and Southern', 'Mozambique': 'Africa Eastern and Southern',
    'Namibia': 'Africa Eastern and Southern', 'Rwanda': 'Africa Eastern and Southern',
    'Seychelles': 'Africa Eastern and Southern', 'Somalia': 'Africa Eastern and Southern',
    'South Africa': 'Africa Eastern and Southern', 'South Sudan': 'Africa Eastern and Southern',
    'Tanzania': 'Africa Eastern and Southern', 'Uganda': 'Africa Eastern and Southern',
    'Zambia': 'Africa Eastern and Southern', 'Zimbabwe': 'Africa Eastern and Southern',
    'Benin': 'Africa Western and Central', 'Burkina Faso': 'Africa Western and Central',
    'Cabo Verde': 'Africa Western and Central', 'Cameroon': 'Africa Western and Central',
    'Central African Republic': 'Africa Western and Central', 'Chad': 'Africa Western and Central',
    'Congo, Dem. Rep.': 'Africa Western and Central', 'Congo, Rep.': 'Africa Western and Central',
    "Cote d'Ivoire": 'Africa Western and Central', 'Equatorial Guinea': 'Africa Western and Central',
    'Gabon': 'Africa Western and Central', 'Gambia, The': 'Africa Western and Central',
    'Ghana': 'Africa Western and Central', 'Guinea': 'Africa Western and Central',
    'Guinea-Bissau': 'Africa Western and Central', 'Liberia': 'Africa Western and Central',
    'Mali': 'Africa Western and Central', 'Mauritania': 'Africa Western and Central',
    'Niger': 'Africa Western and Central', 'Nigeria': 'Africa Western and Central',
    'Sao Tome and Principe': 'Africa Western and Central', 'Senegal': 'Africa Western and Central',
    'Sierra Leone': 'Africa Western and Central', 'Togo': 'Africa Western and Central',
    'Afghanistan': 'South Asia', 'Bangladesh': 'South Asia', 'Bhutan': 'South Asia',
    'India': 'South Asia', 'Maldives': 'South Asia', 'Nepal': 'South Asia',
    'Pakistan': 'South Asia', 'Sri Lanka': 'South Asia',
    'Argentina': 'Latin America & Caribbean', 'Bahamas, The': 'Latin America & Caribbean',
    'Barbados': 'Latin America & Caribbean', 'Belize': 'Latin America & Caribbean',
    'Bolivia': 'Latin America & Caribbean', 'Brazil': 'Latin America & Caribbean',
    'Chile': 'Latin America & Caribbean', 'Colombia': 'Latin America & Caribbean',
    'Costa Rica': 'Latin America & Caribbean', 'Cuba': 'Latin America & Caribbean',
    'Dominican Republic': 'Latin America & Caribbean', 'Ecuador': 'Latin America & Caribbean',
    'El Salvador': 'Latin America & Caribbean', 'Grenada': 'Latin America & Caribbean',
    'Guatemala': 'Latin America & Caribbean', 'Guyana': 'Latin America & Caribbean',
    'Haiti': 'Latin America & Caribbean', 'Honduras': 'Latin America & Caribbean',
    'Jamaica': 'Latin America & Caribbean', 'Mexico': 'Latin America & Caribbean',
    'Nicaragua': 'Latin America & Caribbean', 'Panama': 'Latin America & Caribbean',
    'Paraguay': 'Latin America & Caribbean', 'Peru': 'Latin America & Caribbean',
    'St. Kitts and Nevis': 'Latin America & Caribbean', 'St. Lucia': 'Latin America & Caribbean',
    'St. Vincent and the Grenadines': 'Latin America & Caribbean', 'Suriname': 'Latin America & Caribbean',
    'Trinidad and Tobago': 'Latin America & Caribbean', 'Uruguay': 'Latin America & Caribbean',
    'Venezuela, RB': 'Latin America & Caribbean',
    'Albania': 'Europe & Central Asia', 'Armenia': 'Europe & Central Asia',
    'Austria': 'Europe & Central Asia', 'Azerbaijan': 'Europe & Central Asia',
    'Belarus': 'Europe & Central Asia', 'Belgium': 'Europe & Central Asia',
    'Bosnia and Herzegovina': 'Europe & Central Asia', 'Bulgaria': 'Europe & Central Asia',
    'Croatia': 'Europe & Central Asia', 'Cyprus': 'Europe & Central Asia',
    'Czechia': 'Europe & Central Asia', 'Denmark': 'Europe & Central Asia',
    'Estonia': 'Europe & Central Asia', 'Finland': 'Europe & Central Asia',
    'France': 'Europe & Central Asia', 'Georgia': 'Europe & Central Asia',
    'Germany': 'Europe & Central Asia', 'Greece': 'Europe & Central Asia',
    'Hungary': 'Europe & Central Asia', 'Iceland': 'Europe & Central Asia',
    'Ireland': 'Europe & Central Asia', 'Italy': 'Europe & Central Asia',
    'Kazakhstan': 'Europe & Central Asia', 'Kosovo': 'Europe & Central Asia',
    'Kyrgyz Republic': 'Europe & Central Asia', 'Latvia': 'Europe & Central Asia',
    'Lithuania': 'Europe & Central Asia', 'Luxembourg': 'Europe & Central Asia',
    'Malta': 'Europe & Central Asia', 'Moldova': 'Europe & Central Asia',
    'Montenegro': 'Europe & Central Asia', 'Netherlands': 'Europe & Central Asia',
    'North Macedonia': 'Europe & Central Asia', 'Norway': 'Europe & Central Asia',
    'Poland': 'Europe & Central Asia', 'Portugal': 'Europe & Central Asia',
    'Romania': 'Europe & Central Asia', 'Russian Federation': 'Europe & Central Asia',
    'Serbia': 'Europe & Central Asia', 'Slovak Republic': 'Europe & Central Asia',
    'Slovenia': 'Europe & Central Asia', 'Spain': 'Europe & Central Asia',
    'Sweden': 'Europe & Central Asia', 'Switzerland': 'Europe & Central Asia',
    'Tajikistan': 'Europe & Central Asia', 'Turkiye': 'Europe & Central Asia',
    'Turkmenistan': 'Europe & Central Asia', 'Ukraine': 'Europe & Central Asia',
    'United Kingdom': 'Europe & Central Asia', 'Uzbekistan': 'Europe & Central Asia',
    'Australia': 'East Asia & Pacific', 'Brunei Darussalam': 'East Asia & Pacific',
    'Cambodia': 'East Asia & Pacific', 'China': 'East Asia & Pacific',
    'Fiji': 'East Asia & Pacific', 'Indonesia': 'East Asia & Pacific',
    'Japan': 'East Asia & Pacific', 'Kiribati': 'East Asia & Pacific',
    'Korea, Rep.': 'East Asia & Pacific', 'Lao PDR': 'East Asia & Pacific',
    'Malaysia': 'East Asia & Pacific', 'Marshall Islands': 'East Asia & Pacific',
    'Micronesia, Fed. Sts.': 'East Asia & Pacific', 'Mongolia': 'East Asia & Pacific',
    'Myanmar': 'East Asia & Pacific', 'Nauru': 'East Asia & Pacific',
    'New Zealand': 'East Asia & Pacific', 'Palau': 'East Asia & Pacific',
    'Papua New Guinea': 'East Asia & Pacific', 'Philippines': 'East Asia & Pacific',
    'Samoa': 'East Asia & Pacific', 'Singapore': 'East Asia & Pacific',
    'Solomon Islands': 'East Asia & Pacific', 'Thailand': 'East Asia & Pacific',
    'Timor-Leste': 'East Asia & Pacific', 'Tonga': 'East Asia & Pacific',
    'Tuvalu': 'East Asia & Pacific', 'Vanuatu': 'East Asia & Pacific',
    'Viet Nam': 'East Asia & Pacific'
}


region_colors = {
    "Africa Eastern and Southern": "#636efa",
    "Africa Western and Central": "#ef553b",
    "East Asia & Pacific": "#00cc96",
    "Europe & Central Asia": "#ab63fa",
    "Latin America & Caribbean": "#ffa15a",
    "South Asia": "#19d3f3"
}

# Gemiddelde berekenen
avg_life = lifeexp_df.groupby("REF_AREA_NAME")["OBS_VALUE"].mean().reset_index()
avg_life["Region"] = avg_life["REF_AREA_NAME"].map(region_mapping)
avg_life["Color"] = avg_life["Region"].map(region_colors)
avg_life_clean = avg_life.dropna(subset=["Region", "Color"])

# Lijsten initialiseren
buttons = []
region_bars = []
region_names = sorted(avg_life_clean["Region"].unique())

# 1. "Alle Landen"
all_countries = avg_life_clean.sort_values("Region")
trace_all = go.Bar(x=all_countries["REF_AREA_NAME"], y=all_countries["OBS_VALUE"], marker_color=all_countries["Color"], name="Alle Landen")
region_bars.append(trace_all)
buttons.append(dict(
    label="Alle Landen",
    method="update",
    args=[{"visible": [True] + [False] * (len(region_names) + 1)},
          {"title": "Gemiddelde levensverwachting per land"}]
))

# 2. "Alle Regio's"
avg_regions = avg_life_clean.groupby("Region")["OBS_VALUE"].mean().reset_index()
avg_regions = avg_regions.sort_values("OBS_VALUE", ascending=False)
avg_regions["Color"] = avg_regions["Region"].map(region_colors)
bar_regions = go.Bar(x=avg_regions["Region"], y=avg_regions["OBS_VALUE"], marker_color=avg_regions["Color"], name="Regio's")
region_bars.append(bar_regions)
buttons.append(dict(
    label="Alle Regio's",
    method="update",
    args=[{"visible": [False, True] + [False]*len(region_names)},
          {"title": "Gemiddelde levensverwachting per regio"}]
))

# 3. Per regio
for region in region_names:
    region_data = avg_life_clean[avg_life_clean["Region"] == region]
    trace = go.Bar(x=region_data["REF_AREA_NAME"], y=region_data["OBS_VALUE"], marker_color=region_data["Color"], name=region)
    region_bars.append(trace)
    visible = [False] * (len(region_names) + 2)
    visible[region_names.index(region) + 2] = True
    buttons.append(dict(
        label=region,
        method="update",
        args=[{"visible": visible},
              {"title": f"Gemiddelde levensverwachting in {region}"}]
    ))

# Plotten
fig = go.Figure(region_bars)
fig.update_layout(
    title="Gemiddelde levensverwachting per land",
    xaxis_title="Land/Regio",
    yaxis_title="Gemiddelde levensverwachting",
    width=1200,
    height=600,
    paper_bgcolor="#FFF9E3",
    plot_bgcolor="#FFF9E3",
    font=dict(family="Arial", size=13),
    updatemenus=[
        dict(
            type="dropdown",
            direction="down",
            showactive=True,
            x=0.5,
            xanchor="center",
            y=1.2,
            buttons=buttons
        )
    ]
)

fig.add_annotation(
    text="Deze visualisatie toont de gemiddelde levensverwachting van mannen per land of regio, gebaseerd op Wereldbankdata.",
    xref="paper", yref="paper",
    x=0, y=-0.66,
    showarrow=False,
    font=dict(size=12),
    align="center"
)

fig.add_annotation(
    text="Met behulp van het dropdownmenu kun je wisselen tussen alle landen, alle regio’s of afzonderlijke regio's.",
    xref="paper", yref="paper",
    x=0, y=-0.7,
    showarrow=False,
    font=dict(size=12),
    align="center"
)


fig.show()


Bij het selecteren van regio’s zoals Europe & Central Asia, vallen meteen grote onderlinge verschillen op. Landen als Spanje, Italië en Zweden hebben een levensverwachting boven de 80 jaar, terwijl andere landen in dezelfde regio, zoals Oekraïne, Georgië of Moldavië, soms 10 tot 15 jaar lager scoren. Dit is opvallend, omdat deze landen in veel internationale analyses worden samengevoegd als “hoge-inkomensregio”.

De data toont dus dat een vergelijkbaar regionaal of economisch profiel geen garantie biedt voor gelijke gezondheidsuitkomsten. De verschillen wijzen op dieperliggende factoren zoals kwaliteit van zorg, gezondheidsbeleid, sociale ongelijkheid, leefstijl. Dit ondermijnt het idee dat economische groei automatisch leidt tot een gezonde bevolking, zelfs binnen relatief gelijke regio’s.

Andere regio’s vertonen gelijkaardige patronen: landen met vergelijkbaar GDP laten uiteenlopende levensverwachtingen zien. Deze observaties ondersteunen de stelling dat economische indicatoren slechts een deel van het gezondheidsverhaal vertellen.

**Dalende sterfte als motor van langer leven**

Sterftecijfers zijn een directe indicator van volksgezondheid: hoe lager de sterfte, hoe groter de kans dat mensen een hogere leeftijd bereiken. In deze visualisatie verkennen we hoe sterftecijfers zich verhouden tot levensverwachting, en of we een patroon kunnen ontdekken dat wijst op structurele vooruitgang op wereldschaal.

De grafiek toont een dynamisch overzicht van landen, waarbij de positie van elke bubbel het sterftecijfer en de levensverwachting weergeeft, terwijl de grootte en kleur aanvullende context bieden zoals bevolkingsomvang of GDP.

In [ ]:
import pandas as pd
import plotly.express as px

# Data inladen
lifeexp_df = pd.read_csv("lifeexp.csv")
mortality_df = pd.read_csv("mortality.csv")

# Filter: alleen mannen
lifeexp_df = lifeexp_df[
    (lifeexp_df["SEX_NAME"] == "Male") &
    (lifeexp_df["UNIT_MEASURE_NAME"].str.lower().str.contains("year"))
]
mortality_df = mortality_df[mortality_df["SEX_LABEL"] == "Male"]

# Gemiddelde per land en jaar
lifeexp_avg = lifeexp_df.groupby(["REF_AREA_NAME", "TIME_PERIOD"], as_index=False)["OBS_VALUE"].mean()
mortality_avg = mortality_df.groupby(["REF_AREA_LABEL", "TIME_PERIOD"], as_index=False)["OBS_VALUE"].mean()

# Hernoem kolommen voor merge
lifeexp_avg.columns = ["Country", "Year", "OBS_VALUE_lifeexp"]
mortality_avg.columns = ["Country", "Year", "OBS_VALUE_mortality"]

# Merge datasets
bubble_df = pd.merge(lifeexp_avg, mortality_avg, on=["Country", "Year"])
bubble_df = bubble_df.dropna(subset=["OBS_VALUE_lifeexp", "OBS_VALUE_mortality"])

# Bubble chart maken
fig = px.scatter(
    bubble_df,
    x="OBS_VALUE_mortality",
    y="OBS_VALUE_lifeexp",
    animation_frame="Year",
    animation_group="Country",
    size="OBS_VALUE_mortality",
    color="OBS_VALUE_lifeexp",
    hover_name="Country",
    size_max=60,
    color_continuous_scale="Plasma",
    range_color=[40, 85],
    title="Levensverwachting vs. Sterftecijfer per land per jaar (mannen)",
    labels={
        "OBS_VALUE_lifeexp": "Levensverwachting (jaren)",
        "OBS_VALUE_mortality": "Sterfte per 1.000 mannen"
    }
)

# Layout aanpassen
fig.update_layout(
    width=1000,
    height=600,
    xaxis_title="Sterfte per 1.000 mannen",
    yaxis_title="Levensverwachting (jaren)",
    title_font_size=22,
    dragmode=False,
    paper_bgcolor="#FFF9E3",
    plot_bgcolor="#FFF9E3",
    font=dict(family="Arial", size=13)
)

# Sliderstijl aanpassen
if "sliders" in fig.layout:
    fig.layout.sliders[0].bgcolor = "#FFEEC0"
    fig.layout.sliders[0].currentvalue.font.color = "#333"
    fig.layout.sliders[0].currentvalue.font.size = 14

fig.add_annotation(
    text="Deze animatie toont de relatie tussen sterftecijfers en levensverwachting van mannen per land, van 1960 tot 2022.",
    xref="paper", yref="paper",
    x=0.2, y=-0.22,
    showarrow=False,
    font=dict(size=12),
    align="center"
)

fig.add_annotation(
    text="Landen met een lagere sterfte vertonen meestal een hogere levensverwachting, al varieert dit sterk per jaar en regio.",
    xref="paper", yref="paper",
    x=0.2, y=-0.27,
    showarrow=False,
    font=dict(size=12),
    align="center"
)

fig.show()


In de scatterplot zien we een opvallende tendens: landen met lage sterftecijfers hebben vrijwel altijd een hogere levensverwachting. De meeste van deze landen bevinden zich bovendien aan de rechterkant van de grafiek, wat duidt op een sterker ontwikkeld gezondheidszorgsysteem. Deze groep wordt grotendeels gevormd door landen met een relatief hoog GDP per capita, wat suggereert dat economische groei en gezondheidsverbetering elkaar versterken.

Dit illustreert een wereldwijde verbetering van volksgezondheid, waarbij economische ontwikkeling een centrale rol lijkt te spelen.

**Waar sterfte hoog blijft, ontbreekt economische vooruitgang**

Een wereldkaart met sterftecijfers laat in één oogopslag zien waar de gezondheidsuitdagingen het grootst zijn. Mannelijke sterfte, in het bijzonder, is sterk beïnvloed door factoren zoals levensstandaard, werkomstandigheden, conflict, toegang tot medische zorg en levensstijl. In deze visualisatie brengen we deze geografische ongelijkheid in kaart.

Door per land het sterftecijfer bij mannen te visualiseren, kunnen we observeren hoe de geografische spreiding samenhangt met economische ontwikkeling. Leidt een hoger welvaartsniveau daadwerkelijk tot lagere sterfte? Of zijn er uitzonderingen die dit patroon doorbreken?

In [ ]:
import pandas as pd
import plotly.express as px

# Data inladen
mortality_df = pd.read_csv("mortality.csv")

# Filter op mannen en tot en met 2022
mortality_men = mortality_df[(mortality_df["SEX_LABEL"] == "Male") & (mortality_df["TIME_PERIOD"] <= 2022)]

# Gemiddelde per land en jaar
mortality_yearly_avg = mortality_men.groupby(["REF_AREA_LABEL", "TIME_PERIOD"], as_index=False)["OBS_VALUE"].mean()

# Choropleth-kaart met animatie
fig = px.choropleth(
    mortality_yearly_avg,
    locations="REF_AREA_LABEL",
    locationmode="country names",
    color="OBS_VALUE",
    animation_frame="TIME_PERIOD",
    color_continuous_scale="Reds",
    range_color=(0, 1000),
    title="Sterftecijfer bij mannen per land per jaar",
    labels={
        "OBS_VALUE": "Sterfte per 1.000 volwassenen",
        "REF_AREA_LABEL": "Land",
        "TIME_PERIOD": "Jaar"
    }
)

# Oceaan lichtblauw kleuren
fig.update_geos(
    showframe=False,
    showcoastlines=False,
    projection_type="natural earth",
    fitbounds="locations",
    bgcolor="#EAF6FA"
)

# Pastel layout
fig.update_layout(
    dragmode=False,
    width=1200,
    height=700,
    margin=dict(l=20, r=20, t=80, b=100),
    paper_bgcolor="#FFF9E3",
    plot_bgcolor="#FFF9E3",
    title_font_size=24,
    title_x=0.5,
    font=dict(family="Arial", size=13)
)

# Sliderkleur aanpassen (net iets donkerder geel)
if "sliders" in fig.layout:
    fig.layout.sliders[0].bgcolor = "#FFEEC0"
    fig.layout.sliders[0].currentvalue.font.color = "#333333"
    fig.layout.sliders[0].currentvalue.font.size = 14

# Ondertitel boven slider
fig.add_annotation(
    text="Deze kaart toont het sterftecijfer bij mannen wereldwijd per jaar.<br>Gebruik de tijdsslider onderaan om trends door de tijd te bekijken.",
    xref="paper", yref="paper",
    x=0.5, y=-0.07,
    showarrow=False,
    font=dict(size=12),
    align="center"
)

fig.show()

De kaart laat een duidelijke regionale clustering zien. De hoogste sterftecijfers bij mannen worden geregistreerd in Sub-Sahara Afrika, delen van Zuid-Azië en enkele landen in het Midden-Oosten. Deze gebieden worden ook gekenmerkt door een relatief laag GDP per capita, beperkte toegang tot medische infrastructuur en vaak instabiele sociaaleconomische omstandigheden.

In tegenstelling daarmee vertonen Noord-Amerika, West-Europa, Australië en Oost-Azië opvallend lage sterftecijfers. Dit zijn juist de regio’s waar economische ontwikkeling de afgelopen decennia sterk is geweest en waar gezondheidszorgsystemen robuust en breed toegankelijk zijn. De geografische verdeling van sterfte op de kaart ondersteunt daarmee het idee dat economische groei een bepalende factor is voor volksgezondheid.

Hoewel andere factoren (zoals culturele gewoontes of gezondheidsbeleid) uiteraard ook meespelen, toont deze visualisatie aan dat de afwezigheid van economische middelen samenvalt met slechtere gezondheidsuitkomsten. Daarmee levert de kaart visueel bewijs voor het argument dat investeringen in economische groei kunnen bijdragen aan het verlagen van sterfte wereldwijd

**Meer geld, maar niet altijd een langer leven**

Wereldwijde levensverwachting wordt vaak gebruikt als maatstaf voor gezondheid, ontwikkeling en welvaart. En inderdaad: in veel rijke landen ligt de levensverwachting hoog, wat het idee voedt dat economische groei leidt tot betere gezondheidsuitkomsten. Maar wanneer we deze cijfers geografisch in kaart brengen, ontstaan er nuances die deze aanname ter discussie stellen.

In deze visualisatie zien we per land de levensverwachting van mannen. Daarmee krijgen we niet alleen een indruk van de algemene gezondheidstoestand in verschillende delen van de wereld, maar ook de mogelijkheid om te onderzoeken of hoge levensverwachting altijd samengaat met economische voorspoed.

In [ ]:
import pandas as pd
import plotly.express as px

# Data inladen
lifeexp_df = pd.read_csv("lifeexp.csv")

# Filter op mannen en eenheid in jaren
lifeexp_yearly = lifeexp_df[
    (lifeexp_df["SEX_NAME"] == "Male") &
    (lifeexp_df["UNIT_MEASURE_NAME"].str.lower().str.contains("year"))
]

# Gemiddelde per land en jaar
lifeexp_yearly_avg = lifeexp_yearly.groupby(["REF_AREA_NAME", "TIME_PERIOD"], as_index=False)["OBS_VALUE"].mean()

# Choropleth-kaart aanmaken
fig = px.choropleth(
    lifeexp_yearly_avg,
    locations="REF_AREA_NAME",
    locationmode="country names",
    color="OBS_VALUE",
    animation_frame="TIME_PERIOD",
    color_continuous_scale="Blues",
    range_color=(40, 85),
    title="Levensverwachting bij mannen per land per jaar",
    labels={
        "OBS_VALUE": "Levensverwachting (jaren)",
        "REF_AREA_NAME": "Land",
        "TIME_PERIOD": "Jaar"
    }
)

# Oceaan en algemene stijl
fig.update_geos(
    showframe=False,
    showcoastlines=False,
    projection_type="natural earth",
    fitbounds="locations",
    bgcolor="#EAF6FA"  # Lichtblauw
)

# Algemene layout
fig.update_layout(
    dragmode=False,
    width=1200,
    height=700,
    margin=dict(l=20, r=20, t=80, b=100),
    paper_bgcolor="#FFF9E3",
    plot_bgcolor="#FFF9E3",
    title_font_size=24,
    title_x=0.5,
    font=dict(family="Arial", size=13)
)

# Sliderkleur aanpassen (donker pastelgeel)
if "sliders" in fig.layout:
    fig.layout.sliders[0].bgcolor = "#FFEEC0"  # Slider achtergrond
    fig.layout.sliders[0].currentvalue.font.color = "#333"
    fig.layout.sliders[0].currentvalue.font.size = 14

# Ondertitel toevoegen
fig.add_annotation(
    text="Deze kaart toont de levensverwachting bij mannen wereldwijd per jaar.<br>Gebruik de tijdsslider onderaan om trends door de tijd te bekijken.",
    xref="paper", yref="paper",
    x=0.5, y=-0.07,
    showarrow=False,
    font=dict(size=12),
    align="center"
)

fig.show()


Hoewel de kaart bevestigt dat landen met een hoog GDP per capita — zoals Noorwegen, Japan en Australië — inderdaad hoge levensverwachtingen kennen, laat ze ook belangrijke uitzonderingen zien. Zo hebben sommige landen in het Midden-Oosten of Zuid-Amerika, ondanks een redelijk ontwikkelde economie, een levensverwachting die aanzienlijk lager ligt dan die van landen met vergelijkbaar of zelfs lager GDP.

Daarnaast tonen landen als Cuba of Vietnam dat het mogelijk is om met beperkte economische middelen toch relatief hoge levensverwachtingen te bereiken. Zulke voorbeelden suggereren dat andere factoren, zoals universele gezondheidszorg, onderwijs, publieke gezondheidsbeleid of culturele leefstijlen, minstens even bepalend kunnen zijn.

De kaart nodigt uit tot een genuanceerde interpretatie: economische groei is vaak een gunstige voorwaarde, maar geen garantie voor een lang leven. Gezondheid blijkt niet uitsluitend te koop; het vereist ook politieke keuzes, maatschappelijke structuren en gedeelde waarden rond zorg en preventie.

**Levensverwachting groeit, maar de kloof blijft bestaan**

Een wereldwijde stijging in levensverwachting klinkt als een succesverhaal — en in zekere zin is dat het ook. Sinds 1960 is de gemiddelde levensverwachting in bijna alle regio’s aanzienlijk toegenomen. Maar de manier waarop die vooruitgang is verdeeld over de wereld, roept vragen op over ongelijkheid. Want profiteren alle regio’s evenveel van de toegenomen welvaart, medische innovatie en globalisering?

Deze lijnplot toont de ontwikkeling van de levensverwachting van mannen per wereldregio over meer dan zes decennia. Door deze trends te vergelijken, kunnen we structurele verschillen en mondiale gezondheidsongelijkheid blootleggen.

In [ ]:
import pandas as pd
import plotly.graph_objects as go

# CSV-bestanden inladen
mortality_df = pd.read_csv("mortality.csv")
lifeexp_df = pd.read_csv("lifeexp.csv")

# Filter alleen mannen
mortality_df = mortality_df[mortality_df["SEX_LABEL"] == "Male"]
lifeexp_df = lifeexp_df[lifeexp_df["SEX_NAME"] == "Male"]

# Gewenste regio's
desired_regions = [
    "Africa Eastern and Southern",
    "Africa Western and Central",
    "East Asia & Pacific",
    "Europe & Central Asia",
    "Latin America & Caribbean",
    "South Asia",
    "Sub-Saharan Africa"
]

# Filter op regio's
mortality_filtered = mortality_df[mortality_df["REF_AREA_LABEL"].isin(desired_regions)]
lifeexp_filtered = lifeexp_df[lifeexp_df["REF_AREA_NAME"].isin(desired_regions)]

# Figuur maken
fig = go.Figure()

# Pastelkleuren
pastel_colors = ['#5DADE2', '#F5B7B1', '#82E0AA', '#F9E79F', '#D7BDE2', '#AED6F1', '#FADBD8']

# Sterftecijfer toevoegen
for i, region in enumerate(desired_regions):
    df_region = mortality_filtered[mortality_filtered["REF_AREA_LABEL"] == region]
    fig.add_trace(go.Scatter(
        x=df_region["TIME_PERIOD"],
        y=df_region["OBS_VALUE"],
        name=region,
        visible=True,
        mode='lines',
        line=dict(color=pastel_colors[i % len(pastel_colors)], width=3)
    ))

# Levensverwachting toevoegen
for i, region in enumerate(desired_regions):
    df_region = lifeexp_filtered[lifeexp_filtered["REF_AREA_NAME"] == region]
    fig.add_trace(go.Scatter(
        x=df_region["TIME_PERIOD"],
        y=df_region["OBS_VALUE"],
        name=region + " (Life Exp)",
        visible=False,
        mode='lines',
        line=dict(color=pastel_colors[i % len(pastel_colors)], dash='dot', width=3)
    ))

# Zichtbaarheid instellen
n = len(desired_regions)
visibility_mortality = [True]*n + [False]*n
visibility_lifeexp = [False]*n + [True]*n

# Layout
fig.update_layout(
    updatemenus=[dict(
        type="buttons",
        direction="right",
        buttons=[
            dict(label="Sterftecijfer",
                 method="update",
                 args=[{"visible": visibility_mortality},
                       {"title": "Sterftecijfer onder volwassen mannen"}]),
            dict(label="Levensverwachting",
                 method="update",
                 args=[{"visible": visibility_lifeexp},
                       {"title": "Levensverwachting bij geboorte (mannen)"}])
        ],
        x=0.01, xanchor="left", y=1.22, yanchor="top"
    )],
    title={
        "text": " Trends in sterfte en levensverwachting (mannen, per regio)<br><sub>Interactieve visualisatie met pastelstijl (1960–2022)</sub>",
        "x": 0.5,
        "xanchor": "center"
    },
    xaxis=dict(
        title=" Jaar",
        showgrid=True,
        gridcolor='#FDEBD0',
        zeroline=False
    ),
    yaxis=dict(
        title=" Waarde",
        showgrid=True,
        gridcolor='#FDEBD0',
        zeroline=False
    ),
    paper_bgcolor="#FFF9E3",
    plot_bgcolor="#FFF9E3",
    width=1100,
    height=650,
    margin=dict(l=40, r=40, t=100, b=140),
    font=dict(family="Arial", size=13)
)

# Ondertekst
fig.add_annotation(
    text="Gebruik de knoppen om te wisselen tussen sterftecijfers en levensverwachting per wereldregio. Alleen mannen (1960–2022).",
    xref="paper", yref="paper",
    x=0.5, y=-0.3,
    showarrow=False,
    font=dict(size=12),
    align="center"
)

fig.show()



De visualisatie toont een consistente stijging in levensverwachting in bijna alle regio’s sinds 1960. Toch valt onmiddellijk op dat de startpunten en eindniveaus sterk verschillen. Regio’s zoals Europa, Noord-Amerika en Oost-Azië begonnen al met een relatief hoge levensverwachting en zijn sindsdien verder gestegen naar meer dan 75 jaar. Sub-Sahara Afrika daarentegen startte ver onder de 50 jaar en heeft zelfs vandaag de dag nog moeite om boven de 60 jaar uit te komen.

De afstand tussen de best en slechtst presterende regio’s is dus niet fundamenteel kleiner geworden. Hoewel de levensverwachting overal stijgt, blijft de kloof relatief stabiel of groeit zelfs licht in bepaalde periodes. Deze structurele ongelijkheid toont dat economische globalisering niet automatisch leidt tot gelijke gezondheidswinsten voor alle bevolkingen.

Bovendien zien we dat sommige regio’s (zoals Latijns-Amerika en Zuid-Azië) periodes van stagnatie of vertraging kennen, terwijl andere blijven stijgen. Dit wijst erop dat externe factoren zoals conflict, pandemieën, beleid en ongelijkheid een rol blijven spelen ongeacht het globale economische plaatje.

De grafiek onderstreept daarmee het idee dat gezondheidsverbetering geen automatisch neveneffect is van economische groei. Zonder gericht beleid en rechtvaardige verdeling van middelen blijven bepaalde regio’s achter.

**Mondiale vooruitgang maskeert ongelijkheid**

Op wereldschaal lijkt de relatie tussen economische ontwikkeling en volksgezondheid een succesverhaal. Sinds 1960 is de gemiddelde levensverwachting wereldwijd gestegen, terwijl het sterftecijfer gestaag is gedaald. Deze lange termijntrends lijken aan te tonen dat vooruitgang mogelijk is — en dat globalisering, technologische innovatie en economische groei bijdragen aan een betere gezondheid voor miljarden mensen.

Toch werpt deze visualisatie ook een ander licht op het verhaal: hoe eerlijk zijn deze winsten verdeeld? En vertellen wereldgemiddelden wel het hele verhaal?

In [ ]:
import pandas as pd
import plotly.graph_objects as go

# CSV-bestanden inladen
lifeexp_df = pd.read_csv("lifeexp.csv")
mortality_df = pd.read_csv("mortality.csv")

# Filter alleen mannen
lifeexp_men = lifeexp_df[lifeexp_df['SEX_NAME'] == 'Male']
mortality_men = mortality_df[mortality_df['SEX_LABEL'] == 'Male']

# Gemiddelde per jaar
lifeexp_avg = lifeexp_men.groupby("TIME_PERIOD")["OBS_VALUE"].mean()
mortality_avg = mortality_men.groupby("TIME_PERIOD")["OBS_VALUE"].mean()

# Interactieve Plotly-figuur
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=lifeexp_avg.index, y=lifeexp_avg.values,
    mode='lines',
    name='Gemiddelde levensverwachting (mannen)',
    line=dict(color='#5DADE2', width=3)  # pastelblauw, dikker
))

fig.add_trace(go.Scatter(
    x=mortality_avg.index, y=mortality_avg.values,
    mode='lines',
    name='Gemiddeld sterftecijfer (mannen)',
    line=dict(color='#F1948A', width=3)  # pastelrood, dikker
))

# Layout aanpassen
fig.update_layout(
    title={
        'text': " Wereldwijde trend in levensverwachting en sterfte (1960–2022)<br><sub>Gemiddelden voor mannen per jaar wereldwijd</sub>",
        'x': 0.5,
        'xanchor': 'center'
    },
    xaxis=dict(
        title=" Jaar",
        showgrid=True,
        gridcolor='#FDEBD0',
        zeroline=False
    ),
    yaxis=dict(
        title=" Waarde",
        showgrid=True,
        gridcolor='#FDEBD0',
        zeroline=False
    ),
    legend_title=" Indicator",
    width=1000,
    height=600,
    margin=dict(l=40, r=40, t=100, b=160),
    paper_bgcolor="#FFF9E3",
    plot_bgcolor="#FFF9E3",
    font=dict(family="Arial", size=13)
)

# Ondertekst
fig.add_annotation(
    text="Deze grafiek toont hoe de gemiddelde levensverwachting en het sterftecijfer voor mannen wereldwijd zijn veranderd van 1960 tot 2022.",
    xref="paper", yref="paper",
    x=0.5, y=-0.3,
    showarrow=False,
    font=dict(size=12),
    align="center"
)

fig.show()


De grafiek laat overtuigend zien dat de wereldbevolking er qua gezondheid beter voor staat dan zestig jaar geleden. De levensverwachting is gestegen van ongeveer 52 jaar in 1960 naar ruim boven de 70 jaar vandaag de dag. Tegelijkertijd is het wereldwijde sterftecijfer meer dan gehalveerd. Deze trends wijzen op betere gezondheidszorg, hogere levensstandaarden, effectievere preventie en toegang tot medicijnen.

Toch zegt deze algemene vooruitgang weinig over de verdeling van deze winst. Wereldgemiddelden verbergen grote verschillen tussen landen en regio’s. Terwijl sommige bevolkingen profiteren van hoogtechnologische zorg en preventieprogramma’s, sterven anderen nog steeds vroegtijdig aan vermijdbare oorzaken. In die zin vertelt deze grafiek een dubbel verhaal: ja, de mensheid is gezonder dan ooit, maar nee, die gezondheid is niet universeel.

Deze visualisatie is daarom niet per se een argument voor of tegen het belang van economische groei, maar een uitnodiging tot nuancering. Het toont de potentie van vooruitgang, maar roept ook de ethische vraag op: wie blijft achter en waarom?

##Referenties
- Preston. S. (2022). Health, income and the Preston curve: A long view. Journal of Economic Dynamics and Control, 143, 104516. https://doi.org/10.1016/j.ehb.2022.101212
- Lutz, W., & Kebede, E. (2018). Education and health: Redrawing the Preston curve. _Population and Development Review_, 44(2), 343–361. https://doi.org/10.1111/padr.12141
- Niu, X., Yang, Y., & Wang, Y. (2021). Does the Economic Growth Improve Public Health? A Cross-Regional Heterogeneous Study in China. _Frontiers in Public Health_, 9. https://doi.org/10.3389/fpubh.2021.704155
